In [2]:
# 🚗 Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 📚 Librerías
import pandas as pd
import numpy as np
from functools import reduce
import os

# 📌 Cargar la base reducida
ruta_base_reducida = "/content/drive/MyDrive/Datos/7_Base_Modelos_Predictivos_Reducida.parquet"
base = pd.read_parquet(ruta_base_reducida)

print("🔎 Información inicial:")
print(base.info())

# 📌 Asegurar que no haya duplicados por NIT y Año
duplicados = base.duplicated(subset=['NIT', 'Año'], keep=False)
if duplicados.sum() > 0:
    base = base.drop_duplicates(subset=['NIT', 'Año'])
    print(f"✅ Eliminados {duplicados.sum()} duplicados por NIT y Año.")
else:
    print("✅ No se encontraron duplicados.")

# 📌 Seleccionar solo variables numéricas útiles (excluyendo NIT, Año, RQ)
excluir = ['NIT', 'Año', 'RQ']
columnas_numericas = base.select_dtypes(include=['float64', 'int64']).columns
columnas_funcionales = [col for col in columnas_numericas if col not in excluir]

# 📦 Crear tablas funcionales año a año por NIT
tablas_funcionales = {}
for col in columnas_funcionales:
    print(f"📈 Procesando indicador: {col}")
    tabla = base.pivot(index='NIT', columns='Año', values=col)
    tabla = tabla.sort_index(axis=1)
    tabla.columns = [f"{col}_{a}" for a in tabla.columns]
    tablas_funcionales[col] = tabla

# 📦 Crear tabla funcional para RQ
print("📈 Procesando RQ...")
tabla_rq = base.pivot(index='NIT', columns='Año', values='RQ')
tabla_rq = tabla_rq.sort_index(axis=1)
tabla_rq.columns = [f"RQ_{a}" for a in tabla_rq.columns]
tablas_funcionales['RQ'] = tabla_rq

# 🔗 Unir todas las tablas por NIT
print("🔗 Uniendo todas las variables funcionales...")
base_funcional_reducida = reduce(lambda left, right: left.join(right, how='outer'), tablas_funcionales.values())

# 📎 Agregar DEP y CIIU_Letra
base_funcional_reducida['DEP'] = base.groupby('NIT')['DEP'].first()
base_funcional_reducida['CIIU_Letra'] = base.groupby('NIT')['CIIU_Letra'].first()

# ✅ Orden final
base_funcional_reducida.sort_index(inplace=True)

print(f"\n✅ Base funcional reducida creada con forma: {base_funcional_reducida.shape}")

# 💾 Guardar base funcional reducida para k-NN funcional
ruta_guardar = "/content/drive/MyDrive/Datos/8_Base_Funcional_Reducida_RQ.parquet"
base_funcional_reducida.to_parquet(ruta_guardar)
print(f"💾 Guardada en: {ruta_guardar}")
# 📝 Crear resumen automático
ruta_resumen = "/content/drive/MyDrive/Datos/8_resumen.txt"
with open(ruta_resumen, "w") as f:
    f.write("📄 Resumen de creación de base funcional reducida\n")
    f.write("===============================================\n\n")
    f.write("✅ Base funcional generada para el modelo k-NN funcional.\n")
    f.write(f"📦 Archivo generado: 8_Base_Funcional_Reducida_RQ.parquet\n")
    f.write(f"👥 Número de empresas (NIT únicos): {base_funcional_reducida.shape[0]}\n")
    f.write(f"📊 Número total de columnas: {base_funcional_reducida.shape[1]}\n\n")

    f.write("🔢 Indicadores funcionales procesados:\n")
    for col in columnas_funcionales:
        f.write(f"  - {col}\n")

    f.write("\n📌 También se incluyeron:\n")
    f.write("  - Variable RQ (una por año)\n")
    f.write("  - Variable DEP (primer valor por NIT)\n")
    f.write("  - Variable CIIU_Letra (primer valor por NIT)\n\n")

    f.write("💾 Ruta de guardado:\n")
    f.write("  /content/drive/MyDrive/Datos/8_Base_Funcional_Reducida_RQ.parquet\n")

print(f"📝 Resumen guardado en: {ruta_resumen}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔎 Información inicial:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52575 entries, 0 to 52574
Data columns (total 22 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   RA          52575 non-null  float64
 1   RCID        52575 non-null  float64
 2   RAF         52575 non-null  float64
 3   MO          52575 non-null  float64
 4   PPP         52575 non-null  float64
 5   LG          52575 non-null  float64
 6   T           52575 non-null  float64
 7   ROE         52575 non-null  float64
 8   KTNO        52575 non-null  float64
 9   MB          52575 non-null  float64
 10  AFT         52575 non-null  float64
 11  RI          52575 non-null  float64
 12  RCC         52575 non-null  float64
 13  ROI         52575 non-null  float64
 14  RAO         52575 non-null  float64
 15  PPI         52575 non-null  float64
 